In [4]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from src.data_processing import load_data, basic_info

# Wczytanie danych
df = load_data('../data/SMSSpamCollection')

# Podstawowe informacje
basic_info(df)

Pierwsze 5 wiadomości:
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

Statystyki labeli:
label
ham     4825
spam     747
Name: count, dtype: int64

Informacje o danych:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


In [1]:
import pandas as pd
print(pd.__version__)

2.2.3
